# Atividade Prática 5

## Convolução linear e sinais bi-infinitos

### Entrega: até 29/05/2020 às 23:59 no e-disciplinas

#### Nome: ________________________ N° USP: ________  ( ) Grad ( ) Pós

---
###  Revisão: convolução circular

Vimos na seção 4.2 a definição de convolução circular entre dois sinais $x,h\in\mathbb{C}^N$ como o sinal $y=x*h\in\mathbb{C}^N$ definido pela equação
$$y_n = \sum_{m=0}^{N-1}h_mx_{n-m}.$$

Essa equação pressupõe que $x$ e $h$ são estendidos de forma *periódica*, ou seja, são reinterpretados como sinais de tempo bi-infinito ($n\in\mathbb{Z}$) e satisfazem
$$x_{n+kN} \longleftarrow x_n,\quad h_{n+kN}\longleftarrow h_n,\quad\forall n\in\{0,1,\ldots,N-1\},\ \forall k\in\mathbb{Z}\backslash\{0\},$$
sendo que o resultado $y$, consequentemente, também é bi-infinto e periódico ($y_{n+kN} = y_n,\ \forall n,k$).

###  Convolução linear

A *convolução linear* parte da premissa de que a extensão de *sinais de duração finita* para fora de seu intervalo de indexação usual segue a regra do *completamento por zeros*: novamente todos os sinais são reinterpretados como de tempo bi-infinito, porém
$$x_n \longleftarrow 0,\ \forall n\not\in I(x),$$
onde $I(x)$ é o conjunto finito dos índices onde $x$ está definido; a mesma extensão é aplicada a $h$ ($h_n\longleftarrow 0,\ \forall n\not\in I(h)$). Com essa extensão, a convolução linear $x\overline*h$ é definida como o sinal $y$ tal que
$$y_n = \sum_{m=-\infty}^{\infty}h_mx_{n-m} = \sum_{m\in I(h)}h_mx_{n-m} \left( = \sum_{m=-\infty}^{\infty}x_mh_{n-m} = \sum_{m\in I(x)}x_mh_{n-m}\right),$$
onde a soma acima é sempre finita, uma vez que tanto $x$ quanto $h$ só podem ser $\neq 0$ nos intervalos finitos $I(x)$ e $I(h)$, respectivamente. Apesar de $y_n$ também estar definido $\forall n\in\mathbb{Z}$, computacionalmente estamos interessados apenas na faixa de índices onde a equação de convolução inclui índices em $I(h)$ e $I(x)$, de tal forma que para todos os efeitos práticos $y$ também pode ser considerado como um sinal de duração finita.

Essa definição traz uma grande flexibilidade no tratamento de sinais, pois permite a convolução de sinais de durações diferentes, bem como definidos em intervalos de índices arbitrários. Isso é especialmente conveniente no processamento de sinais $x$ de comprimento muito grande (ou desconhecido a priori, como no processamento em tempo-real) por filtros de convolução que possuem poucos "taps" (coeficientes não-nulos), onde a formulação com o somatório sobre $m\in I(h)$ corresponde à implementação mais eficiente.

#### Exemplo: filtro da média centralizada de 5 amostras

Considere o filtro dado pela equação
$$y_n = \frac{1}{5}\big(x_{n-2}+x_{n-1}+x_n+x_{n+1}+x_{n+2}\big).$$
Esse filtro pode ser aplicado em sinais $x$ de comprimento arbitrário e corresponde à convolução linear com o vetor
$$h=\Bigg(\frac{1}{5},\frac{1}{5},\overbrace{\frac{1}{5}}^{n=0},\frac{1}{5},\frac{1}{5}\Bigg)$$
(note que o vetor $h$ acima é definido para os índices $n=-2,-1,0,1,2$). A tabela abaixo ilustra a aplicação desse filtro em um vetor $x\in\mathbb{C}^N$ com $x_n=1,\ n=0,\ldots,N-1$, considerando o completamento por zeros:

$$\begin{array}{r|rrrr:rrrrrrr:rrrr}
  n & \cdots & -3 & -2 & -1 & 0 & 1 & 2 & \cdots & N-3 & N-2 & N-1 & N & N+1 & N+2 & \cdots \\\hline
x_n & \cdots &  0 &  0 &  0 & 1 & 1 & 1 & \cdots &   1 &   1 &   1 & 0 &   0 &   0 & \cdots\\
y_n & \cdots &  0 &  \frac{1}{5} & \frac{2}{5} & \frac{3}{5} & \frac{4}{5} & 1 & \cdots &
1 & \frac{4}{5} & \frac{3}{5} & \frac{2}{5} & \frac{1}{5} & 0 & \cdots
\end{array}$$

### Tamanho resultante da convolução linear

Note que se $x$ tem tamanho $N$, a convolução linear $y=x\overline*h$ geralmente terá um trecho não-nulo de comprimento *maior* do que $N$, dependendo de quantos coeficientes o filtro possui. Ao considerar o vetor $h'$ (dos coeficientes do filtro revertidos no tempo) como uma janela deslizante aplicada sobre o sinal $x$, não é difícil de ver que se $x$ é um vetor de tamanho $N$ e $h$ é um vetor de tamanho $M$, então a parte não-nula de $y$ terá no máximo tamanho $N+M-1$:

$$\begin{array}{l}
\hspace{1.7cm}\overbrace{\square\square\square\square\square\square\square\square\cdots\square\square\square\square\square\square\square\square\square\square\square\square\square\square\square\square}^{x}\\
\hspace{1.75cm}\vdots\hspace{6.7cm}\vdots\\
\underbrace{\square\square\square\square\square\square\square}_{h'}\hspace{6.7cm}\underbrace{\square\square\square\square\square\square\square}_{h'}
\end{array}$$

No exemplo do filtro da média de 5 amostras, onde $M=5$, a saída $y$ tem $N+4$ coeficientes não-nulos, que correspondem ao intervalo $n=-2,-1,0,1,\ldots,N-1,N,N+1$.

In [ ]:
# importa dependências
import math as m
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft

---

**Exercício 1:** 

**(a)** Escreva uma função <tt>convolucao_linear(x,h)</tt> que devolve um vetor $y$ de comprimento <tt>len(x)+len(h)-1</tt> correspondendo à convolução linear entre $x$ e $h$. Você deve implementar a convolução diretamente a partir da definição.

**Dica:** *você pode acrescentar zeros (sentinelas) à esquerda e à direita do vetor $x$ para facilitar a implementação; use a função de biblioteca <tt>np.convolve</tt> para comparar os resultados e depurar sua função, testando-as por exemplo no exemplo acima do filtro da média de 5 pontos com um vetor $x$ de tamanho 10.*

**(b)** Use sua função <tt>convolucao_linear(x,h)</tt> para aplicar um filtro da média centralizada de $M=31$ amostras $\left(y_n=\frac{1}{31}\displaystyle\sum_{m=-15}^{+15}x_{n-m}\right)$ ao sinal $x_n=\displaystyle\sum_{k=1}^{K}\frac{1}{k}\sin(2\pi Lkn/N)$ para $K=5$, $L=5$, $N=1000$ e $n=0,1,\ldots,N-1$, e plote o sinal original e o sinal filtrado em um mesmo gráfico, alinhados no tempo.

**Dica:** *o sinal $y$ estará definido para $n=-15,\ldots,1014$.*

In [ ]:
# Resposta do exercício 1

# begin gabarito

# item (a)
def convolucao_linear(x,h):
    M,N = len(h),len(x)
    xx = np.pad(x, (M-1, M-1))
    h_dash = np.flip(h)
    return [ sum(h_dash*xx[m:m+M]) for m in range(N+M-1) ]

# parte opcional: comparar com a implementação de referência:
N = 10; x = np.ones(N)
h = np.ones(5)/5
y = convolucao_linear(x,h)
yy = np.convolve(x,h)
assert (np.linalg.norm(y-yy)/np.linalg.norm(y)<1e-12)

# item (b)
K,L,M,N = 5,5,31,1000
n = np.array(range(N))
x = sum([(1/k)*np.sin(2*m.pi*L*k*n/N) for k in range(1,K+1)])
h = np.ones(M)/M
y = convolucao_linear(x,h)
plt.plot(range(N),x,label='x')
plt.plot(range(-(M//2),N+M//2),y,label="y")
plt.legend()
plt.show()

# end gabarito


### Relacionando a convolução linear com a convolução circular

No diagrama que ilustra a convolução através de uma janela deslizante

$$\begin{array}{l}
\hspace{1.7cm}\overbrace{\square\square\square\square\square\square\square\square\cdots\square\square\square\square\square\square\square\square\square\square\square\square\square\square\square\square}^{x}\\
\hspace{1.75cm}\vdots\hspace{6.7cm}\vdots\\
\underbrace{\square\square\square\square\square\square\square}_{h'}\hspace{6.7cm}\underbrace{\square\square\square\square\square\square\square}_{h'}
\end{array}$$

é fácil ver que a diferença entre a convolução circular e a convolução linear aparece apenas nas extremidades do vetor $x$, pois é ali que aparecem as diferenças entre as extensões periódica e por zeros: na convolução circular, os primeiros valores da entrada ($x_0,x_1,\ldots$) podem ser combinados com valores do fim do vetor ($x_{-1}=x_{N-1},x_{-2}=x_{N-2},\ldots$), ao passo que na convolução linear os valores "ausentes" são tratados como $x_{-1}=x_{-2}=\cdots=0$; uma diferença semelhante entre as duas convoluções ocorre quando a equação usa valores $x_n$ após o final do vetor $x$.

Por outro lado, quando os coeficientes do filtro multiplicam apenas valores $x_{n-m}$ pertencentes ao vetor $x$ original na expressão $\displaystyle\sum_{m\in I(h)}h_mx_{n-m}$, o modo de extensão do vetor (periódica ou por zeros) não faz nenhuma diferença na fórmula.

Isso sugere duas observações:

- que a convolução circular $x*h$ produz os mesmos valores da convolução linear $x\overline*h$ no "interior" do vetor, e que portanto seria possível obter a primeira através da segunda através de uma "pequena" extensão periódica do vetor $x$, copiando um certo número $p$ de amostras do final do vetor ($x_{\tiny\mbox{FIM}}$) como salvaguarda à esquerda, e um certo número $q$ de amostras do início do vetor ($x_{\tiny\mbox{INÍCIO}}$) como salvaguarda à direita do vetor $x$:

$$\begin{array}{l}
\overbrace{\square\square\square\square\square\square}^{x_{\tiny\mbox{FIM}}}\overbrace{\square\square\square\square\square\square\square\square\square\square\cdots\square\square\square\square\square\square\square\square\square\square\square\square\square\square}^{x}\overbrace{\square\square\square\square\square\square}^{x_{\tiny\mbox{INÍCIO}}}\\
\hspace{2cm}\vdots\hspace{6.1cm}\vdots\\
\underbrace{\square\square\square\square\square\square\square\square}_{h'}\hspace{6.1cm}\underbrace{\square\square\square\square\square\square\square\square}_{h'}
\end{array}$$

> O número exato de amostras de salvaguarda de cada lado depende essencialmente do menor índice negativo $x_{-p}$ que será acessado no cômputo de $y_0$ pela convolução circular, e do maior índice $x_{N-1+q}$ além de $N-1$ que será acessado no cômputo de $y_{N-1}$; os valores de $p$ e $q$ não dependem exclusivamente de $M=\mbox{len}(h)$, mas podem ser calculados como $p=\max I(h)$ e $q=|\min I(h)|$, onde $h=(h_{-q},\ldots,h_p)$ (estamos supondo que $0\in I(h)$). No caso dos filtros de médias centralizadas com $M$ amostras ($M$ ímpar) dos exemplos acima, pode-se ver que $p=q=M//2$.

> **Observe** que, a rigor, a convolução circular $x*h$ foi definida apenas entre vetores de mesma dimensão $N$, o que exigiria que $h$ fosse considerado como vetor de $\mathbb{R}^N$ através do acréscimo de $N-M$ zeros, exatamente como foi feito com os exemplos do filtro da média no livro-texto. Entretanto, essa observação só é necessária aqui para deixar clara a notação $x*h$, pois esse aumento do vetor $h$ não é necessário para o cálculo da convolução circular $x*h$ através da convolução linear $x\overline*h$, sendo inclusive indesejável, já que o uso do $h$ original permite maior economia computacional, ao avaliar a equação de convolução considerando apenas os coeficientes não-nulos do filtro.

> **Observe também** que a convolução linear entre o vetor estendido $(x_{\tiny\mbox{FIM}},x,x_{\tiny\mbox{INÍCIO}})$ e o filtro $h$ terá tamanho maior do que a convolução circular desejada, que deveria ter o mesmo tamanho $N$ do vetor $x$ original. Observando o diagrama acima não é difícil verificar que os $N$ valores correspondentes à convolução circular x*h desejada são encontrados a partir da posição $M-1$ da saída da convolução linear $(x_{\tiny\mbox{FIM}},x,x_{\tiny\mbox{INÍCIO}})\overline*h$.

- que o resultado da convolução linear poderia ser obtido por uma convolução circular, desde o vetor $x$ fosse salvaguardado com um número "suficiente" de zeros à direita:

$$\begin{array}{l}
\color{gray}{00000000}\overbrace{\square\square\square\square\square\square\square\square\cdots\square\square\square\square\square\square\square\square\square\square\square\square\square\square\square\square}^{x}\overbrace{00000000}^{0_{M-1}}\\
\hspace{1.75cm}\vdots\hspace{6.7cm}\vdots\\
\underbrace{\square\square\square\square\square\square\square}_{h'}\hspace{6.7cm}\underbrace{\square\square\square\square\square\square\square}_{h'}
\end{array}$$

> Acrescentando $M-1$ zeros ao final do vetor $x$, além de atingirmos o tamanho correto da convolução linear $x\overline*h$, também garantimos que a convolução circular $(x,0_{M-1})*h$ irá considerar, por periodicidade, que são nulos os valores "à esquerda" utilizados nos primeiros resultados computados. Observe que estes zeros à esquerda, indicados em cinza no diagrama acima, não são explicitamente acrescentados ao vetor $x$, mas são considerados nulos pela circularidade implícita no modelo de Fourier.

> Nesse modo de cálculo da convolução linear $x\overline*h$ através da convolução circular $(x,0_{M-1})*h$, não é relevante conhecer o valor de $p$ ou $q$ indicando a posição temporal dos coeficientes do filtro $h$, pois a primeira amostra relevante da saída do filtro de convolução linear corresponde ao alinhamento do vetor $h'$ em relação ao vetor $x$ exatamente como indicado à esquerda no diagrama. Isso é compatível com a observação de que a convolução linear $x\overline*h$ não depende de $p$ ou $q$, como vimos na implementação do exercício 1. Sem qualquer perda de generalidade, pode-se considerar que o filtro é indexado como $h=(h_0,\ldots,h_{M-1})$ e simplesmente completá-lo com $N-1$ zeros para compatibilizar os tamanhos dos dois vetores antes de computar a convolução circular $(x,0_{M-1})*(h,0_{N-1})$, cujo resultado será exatamente igual ao da convolução linear $x\overline*h$ desejada.

---

**Exercício 2:** 

**(a)** Escreva uma função <tt>convolucao_circular(x,h,p)</tt> que calcula a convolução circular $y=x*h\in\mathbb{R}^N$ entre $x\in\mathbb{R}^N$ e o vetor $h=(h_{-q},\ldots,h_p)\in\mathbb{R}^M$ (com $M\le N$), usando sua função <tt>convolucao_linear</tt> e o mecanismo de completamento periódico do vetor $x$ descrito acima. Compare o resultado de sua função com a implementação <tt>conv_circ_fft</tt> fornecida abaixo, aplicando-as aos mesmos sinal e filtro do exercício 1, plotando o gráfico dos sinais original e filtrado, e medindo o erro relativo entre as duas implementações.

**Dica:** *Use um caso pequeno e simples, como o primeiro exemplo com o filtro da média centralizada de 5 amostras, para visualizar os resultados e testar sua implementação. Para compatibilidade entre as implementações, ao usar o a função <tt>conv_circ_fft</tt> os coeficientes do filtro $h$ devem ser reposicionados na forma $(h_0,\ldots,h_{N-1})$, onde os valores $h_{-1},h_{-2},\ldots$ devem ser armazenados nas posições $h_{N-1},h_{N-2},\ldots$.*

**(b)** Escreva uma função <tt>convolucao_linear_fft(x,h)<tt> usando a função <tt>conv_circ_fft</tt> dada, implementando o mecanismo de completamento com zeros à direita descrito acima.

**Dica:** *Não esqueça que para usar a função <tt>conv_circ_fft</tt> os dois vetores precisam ter o mesmo tamanho, que será $N+M-1$ nessa implementação*

In [ ]:
# Resposta do exercício 2

# a função abaixo exige que x e h tenham o mesmo tamanho
def conv_circ_fft(x,h):
    return np.real(ifft(fft(x)*fft(h)))
    
# begin gabarito

# item (a)
def convolucao_circular(x,h,p):
    N,M = len(x),len(h)
    q = M-p-1
    #xx = np.append(x[N-p:N],np.append(x,x[0:q]))
    xx = np.pad(x, (q,p), mode='wrap')
    y = convolucao_linear(xx,h)
    return y[M-1:N+M-1]

# opcional: teste simples
N = 10; x = np.arange(N)
M = 5; h = np.ones(M)/M; p = 2; q = M-p-1
y = convolucao_circular(x,h,p)
hh = np.roll(np.pad(h, (0, N-M)), -p)
yy = conv_circ_fft(x,hh)
assert np.linalg.norm(y-yy)/np.linalg.norm(y)<1e-12

K,L,M,N = 5,5,31,1000
n = np.array(range(N))
x = sum([(1/k)*np.sin(2*m.pi*L*k*n/N) for k in range(1,K+1)])
h = np.ones(M)/M; p = M//2; q = M//2
y = convolucao_circular(x,h,p)
plt.plot(range(N),x,label='x')
plt.plot(range(N),y,label="y")
plt.legend()
plt.show()
hh = np.roll(np.pad(h, (0, N-M)), -p)
yy = conv_circ_fft(x,hh)
print("Erro relativo entre as implementações convolucao_circular e conv_circ_fft:",np.linalg.norm(y-yy)/np.linalg.norm(y))

# item (b)
def convolucao_linear_fft(x,h):
    N,M = len(x),len(h)
    xx = np.pad(x , (0, M-1))
    hh = np.pad(h, (0, N-1))
    y = conv_circ_fft(xx,hh)
    return y

y = convolucao_linear_fft(x,h)
plt.plot(range(N),x,label='x')
plt.plot(range(-(M//2),N+M//2),y,label="y")
plt.legend()
plt.show()
yy = convolucao_linear(x,h)
print("Erro relativo entre as implementações convolucao_linear e convolucao_linear_fft:",np.linalg.norm(y-yy)/np.linalg.norm(y))

# end gabarito